In [1]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup as BS
import re
import ast
import numpy as np
import html.parser

import itertools

import json

#print(fastai.__version__)

In [161]:
def preprocess_sentence(string: str):
    string = html.unescape(string.strip())
    string = re.sub('<[^>]+>', '', string)
    return string

In [138]:
def get_quest_responses(link):
    html_ = urllib.request.urlopen(link).read()
    soup = BS(html_)
    for a in soup.find_all('script',{"src":True}):
        if a['src'].startswith('https'):
            URL = a['src']
    a = soup.find('script',{'src':False,'type':False})
    
    resps = urllib.request.urlopen(URL).read()
    soup = BS(resps,exclude_encodings=["\u00cdn","\u00e3"],)

    a = (soup.find_all()[0].text)#.replace('\\\\', '\\').encode().decode('utf-8')
    a = a.replace("\\\\",'\\')
    a = (bytes(a, 'utf-8').decode("unicode_escape"))
    a = a.strip().split("\n")
    dimensao = preprocess_sentence(a[2])
    empresa = preprocess_sentence(a[3])
    print("    "+empresa+","+dimensao)
    return a,empresa,dimensao
    

In [141]:
def get_respostas(str_list):
    for string in str_list:
        if "aplicar_respostas" in string:
            #regex = r"(?<=\[)([^]]+)(?=\])"
            #matches = re.findall(regex, string)
            return string
    return ""

def norm_answers(answers,quest_len):
    new_ans = {}
    for i in range(quest_len):
        try:
            new_ans[str(i)] = answers[str(i)]
        except:
            new_ans[str(i)] = []
    return new_ans



In [186]:
def checa_subperguntas(pergunta):
    return len(pergunta['sub'])

def get_pergunta(pergunta):
    perg = {}
    alternativas = []
    subpergs = []
    perg['enunciado'] = preprocess_sentence(pergunta['value']['data'][0]['value'])
    #print(pergunta)
    colsize = 0
    perg['colsize'] = 0
    perg['type'] = pergunta['value']['type']
    if pergunta['value']['type'] == 'TableUniqueChoice' or pergunta['value']['type'] == 'TableMultipleChoice':
        for item in pergunta['value']['data']:
            if item['name'] == 'alternative':
                for row in item['value']['row']:
                    colsize=0
                    for col in item['value']['col']:
                        colsize+=1
                        if "(" in col['value']:
                            alternativas.append(preprocess_sentence(row['value']+" - "+col['value']))
                        else:
                            alternativas.append(preprocess_sentence(row['value']+" - ("+col['value']+")"))
                    perg['colsize'] = colsize
    else:
        for alternativa in pergunta['value']['data'][3]['value']:
            alternativas.append(preprocess_sentence(alternativa['value']))
    perg['alternativas'] = alternativas
    if checa_subperguntas(pergunta):
        for subpergunta in pergunta['sub']:
            subpergs += get_pergunta(subpergunta)
    #print(perg)
    return [perg] + subpergs
    

In [209]:
def flat_arr(ans, len_alt):
    ans = np.array(ans,dtype=object)
    dim_size = len_alt/ans.shape[0]
    new_ans = []
    count = 0
    for i,dim in enumerate(ans):
        i *= dim_size
        for n in dim:
            new_ans.append(i+int(n))
    return np.array(new_ans)



In [139]:
anos = ['2014','2015','2016','2017','2018','2019','2020']

# URL = "https://respostas.iseb3.com.br/answer/aHR0cHM6Ly9zaXN0ZW1hLmlzZWIzLmNvbS5ici9xdWVzdGlvbmFyaW9zL2lu%0AZGV4LnBocD9zZWN1cmVVcmw9Q1hOUjQ0a2ZFc2E5LTlsYzVqRFZrQUlzWnA0%0AcXp4RC1KMGkwdFNkbXh3cVlGemEtcGFtdkV3bWo1NUljeDlxZXRHN0k5VmRy%0AYzRvR3JQQk9CZU5mN2JmZ1h6eVlIU0FCV00xaXB6Z0hLNFExUk16T0hvOUNq%0AX3FLekNxdnAzcS1maFZFQjF2N1hEbW1sbHRfeXRqVFFQZ19xRUlsdHVydDFS%0ANGs3b3l5YVpsNkFHeU9vRlJ3YUplcV82anpGUzRNVWo0VC1OTXZicmdsSVRC%0AS1BpS2VHajVycWg3SVhxa2tUcEdxUGtDWVB3akNZZVgxbDNnWUwzd3QwdUhO%0AbmlZUmQxT1FwZF93RGxUTzMzd3FQblBPZ19HLTdySklUQy1qQi1uOUdxYmhW%0AUVRFRmpSNFlUZ1JEYXRCWVpHcFJoQ0pCOUZzY1AzOU9ZVmljSVZQRlp2MklL%0AXzg1bHZLNTdDYnhlelNxREUxUUxZbVc1Sk81SWltWTFCNTZGaWVsb0Vl%0A"
# URL = "https://respostas.iseb3.com.br/cicles/2017"
# html = urllib.request.urlopen(URL).read()

for ano in anos:
    print(ano)
    site = "https://respostas.iseb3.com.br/cicles/"+ano
    site_html = urllib.request.urlopen(site).read()
    soup = BS(site_html)
    links_with_text = []
    for a in soup.find_all('a', href=True): 
        if a.text: 
            links_with_text.append(a['href'])
    for link in links_with_text:
        url2 = 'https://respostas.iseb3.com.br'+link
        page = urllib.request.urlopen(url2).read()
        soup = BS(page)

        links = []
        for a in soup.find_all('iframe'):
            links.append(a['src'])
        resps,empresa,dim = get_quest_responses(links[0])
        resps = get_respostas(resps)
        answers = ast.literal_eval("{"+(re.search('(?<=\{)([^}]+)(?=\})', resps).group(0))+"}")
        new_ans = norm_answers(answers,len(q_flat))

        q = soup.find('script',{'src':False,'type':False})
        q = str(q)
        q = q[q.index('['):]
        q = q[:-11]
        q= json.loads(q)
        q = q[0:4]
        
        qst = []
        for criterio in q:
            #print(html.unescape(criterio['value']['data'][0]['value']))
            for indicador in criterio['sub']:
                print("    "*2+preprocess_sentence(indicador['value']['data'][0]['value']))
                for pergunta in indicador['sub']:
                    qst.append(get_pergunta(pergunta))
        q_flat = list(itertools.chain.from_iterable(qst))
        
        for quest, ans in zip(q_flat,list(new_ans.values())):
            if quest['type'] == "TableUniqueChoice":
                ans = np.array(ans).reshape((-1,1)).tolist()
                #print(quest,ans)    
            ans_vec = np.zeros(len(quest['alternativas']))
            if any(isinstance(el, list) for el in ans):
                ans = flat_arr(ans,len(quest['alternativas']))
            for resposta in ans:
                ans_vec[int(resposta)] = 1
            quest['answer'] = ans_vec.tolist()

    


2014


/scratch/guilhermedrummond/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: invalid escape sequence '\/'
  


    AES TIETE S.A.,Dimensão Geral
    AES TIETE S.A.,Dimensão Natureza do Produto
    AES TIETE S.A.,Dimensão Governança Corporativa
    AES TIETE S.A.,Dimensão Econômico-Financeira
    AES TIETE S.A.,Dimensão Ambiental A
    AES TIETE S.A.,Dimensão Social
    AES TIETE S.A.,Dimensão Mudanças Climáticas
    B2W - COMPANHIA DIGITAL,Dimensão Geral
    B2W - COMPANHIA DIGITAL,Dimensão Natureza do Produto
    B2W - COMPANHIA DIGITAL,Dimensão Governança Corporativa
    B2W - COMPANHIA DIGITAL,Dimensão Econômico-Financeira
    B2W - COMPANHIA DIGITAL,Dimensão Ambiental E
    B2W - COMPANHIA DIGITAL,Dimensão Social
    B2W - COMPANHIA DIGITAL,Dimensão Mudanças Climáticas
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Geral
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Natureza do Produto
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Governança Corporativa
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Econômico-Financeira
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Ambiental IF
    BANCO SANTANDER (BRASIL) 

KeyboardInterrupt: 

In [6]:
# #links_with_text
# site = "https://respostas.iseb3.com.br/cicles/"+"2017"
# site_html = urllib.request.urlopen(site).read()
# soup = BS(site_html)
# links_with_text = []
# for a in soup.find_all('a', href=True): 
#     if a.text: 
#         links_with_text.append(a['href'])

In [142]:
# url2

In [143]:
# url2 = 'https://respostas.iseb3.com.br'+links_with_text[0]

# html = urllib.request.urlopen(url2).read()

# soup = BS(html)

# links = []
# for a in soup.find_all('iframe'):
#     links.append(a['src'])
# links

In [144]:
# links

In [10]:
#pega questionario (a) e respostas (URL)

# html2 = urllib.request.urlopen(links[0]).read()

# soup = BS(html2)
# for a in soup.find_all('script',{"src":True}):
#     if a['src'].startswith('https'):
#         URL = a['src']
# a = soup.find('script',{'src':False,'type':False})

In [145]:
# Processa respostas

# resps = urllib.request.urlopen(URL).read()
# print(URL)
# soup = BS(resps,exclude_encodings=["\u00cdn","\u00e3"],)

# import html.parser

# a = (soup.find_all()[0].text)#.replace('\\\\', '\\').encode().decode('utf-8')
# a = a.replace("\\\\",'\\')
# a = (bytes(a, 'utf-8').decode("unicode_escape"))
# a = a.strip().split("\n")

In [146]:
# precisa do a[2] e a[3]
# dimensao = preprocess_sentence(a[2])
# empresa = preprocess_sentence(a[3])
# print(dimensao,",",empresa)

In [136]:


# new_ans = norm_answers(answers,len(q_flat))
#new_ans

In [140]:
# resps = get_respostas(a)
# answers = ast.literal_eval("{"+(re.search('(?<=\{)([^}]+)(?=\})', resps).group(0))+"}")
# answers

In [195]:
# import html.parser

In [190]:
# soup = BS(html2)
# a = soup.find('script',{'src':False,'type':False})
# a = str(a)
# a = a[a.index('['):]
# a = a[:-11]

In [191]:
# import json

In [192]:
# a= json.loads(a)

In [205]:
# preprocess_sentence(a[0]['sub'][1]['sub'][0]['value']['data'][0]['value'])

In [102]:
#a[0]['sub'][0]['sub'][0]#['value']#['data']#[0]['value']

In [184]:
# a[3]['value']['data'][0]['value']

In [198]:
# a = a[0:4]

In [187]:
# a[0]['sub'][0]['sub'][0]['value']['data'][0]['value']#[0]['value']

In [188]:
# a[0]['sub'][0]['sub'][0]['sub'][3]['value']['data'][0]['value']

In [114]:
# def checa_subperguntas(pergunta):
#     return len(pergunta['sub'])

# def get_pergunta(pergunta):
#     perg = {}
#     alternativas = []
#     subpergs = []
#     perg['enunciado'] = preprocess_sentence(pergunta['value']['data'][0]['value'])
#     #print(pergunta)
#     colsize = 0
#     perg['colsize'] = 0
#     perg['type'] = pergunta['value']['type']
#     if pergunta['value']['type'] == 'TableUniqueChoice' or pergunta['value']['type'] == 'TableMultipleChoice':
#         for item in pergunta['value']['data']:
#             if item['name'] == 'alternative':
#                 for row in item['value']['row']:
#                     colsize=0
#                     for col in item['value']['col']:
#                         colsize+=1
#                         if "(" in col['value']:
#                             alternativas.append(preprocess_sentence(row['value']+" - "+col['value']))
#                         else:
#                             alternativas.append(preprocess_sentence(row['value']+" - ("+col['value']+")"))
#                     perg['colsize'] = colsize
#     else:
#         for alternativa in pergunta['value']['data'][3]['value']:
#             alternativas.append(preprocess_sentence(alternativa['value']))
#     perg['alternativas'] = alternativas
#     if checa_subperguntas(pergunta):
#         for subpergunta in pergunta['sub']:
#             subpergs += get_pergunta(subpergunta)
#     #print(perg)
#     return [perg] + subpergs
    

In [204]:
# qst = []
# for criterio in a:
#     #print(html.unescape(criterio['value']['data'][0]['value']))
#     for indicador in criterio['sub']:
#         print("    "+preprocess_sentence(indicador['value']['data'][0]['value']))
#         for pergunta in indicador['sub']:
#             qst.append(get_pergunta(pergunta))

In [116]:
# import itertools

In [201]:
# q_flat = list(itertools.chain.from_iterable(qst))

In [208]:
# def flat_arr(ans, len_alt):
#     ans = np.array(ans,dtype=object)
#     dim_size = len_alt/ans.shape[0]
#     new_ans = []
#     count = 0
#     for i,dim in enumerate(ans):
#         i *= dim_size
#         for n in dim:
#             new_ans.append(i+int(n))
#     return np.array(new_ans)

# tst = [[0,1,2],[0,1,2],[0,1,2]]
# flat_arr(tst,9)

In [207]:
# tst = np.array([2,2,2,2])
# tst.reshape((-1,1)).tolist()

In [152]:
# for quest, ans in zip(q_flat,list(new_ans.values())):
# #     print(quest,ans)
#     if quest['type'] == "TableUniqueChoice":
#         ans = np.array(ans).reshape((-1,1)).tolist()
#         #print(quest,ans)    
#     ans_vec = np.zeros(len(quest['alternativas']))
#     if any(isinstance(el, list) for el in ans):
#         ans = flat_arr(ans,len(quest['alternativas']))
#     for resposta in ans:
#         ans_vec[int(resposta)] = 1
#     quest['answer'] = ans_vec.tolist()
    

In [206]:
#(len(new_ans),len(q_flat))

In [155]:
#list(answers.values())

In [157]:
#q_flat

In [165]:
#json.dumps(q_flat)
with open('teste.json', 'w') as f:
    json.dump(q_flat, f)
test = pd.read_json('teste.json')

In [213]:
pd.DataFrame(q_flat).drop(['colsize','type'],axis=1).apply(pd.Series.explode).T

,0,0,1,1,1,1,1,1,2,2,...,62,62,62,62,63,63,64,64,64,64
enunciado,O compromisso com o Desenvolvimento Sustentáve...,O compromisso com o Desenvolvimento Sustentáve...,"Se SIM para a PERGUNTA 1, este compromisso est...","Se SIM para a PERGUNTA 1, este compromisso est...","Se SIM para a PERGUNTA 1, este compromisso est...","Se SIM para a PERGUNTA 1, este compromisso est...","Se SIM para a PERGUNTA 1, este compromisso est...","Se SIM para a PERGUNTA 1, este compromisso est...","Se SIM para a PERGUNTA 1, assinale os temas ex...","Se SIM para a PERGUNTA 1, assinale os temas ex...",...,"Se SIM para a PERGUNTA 29, o relatório foi:","Se SIM para a PERGUNTA 29, o relatório foi:","Se SIM para a PERGUNTA 29, o relatório foi:","Se SIM para a PERGUNTA 29, o relatório foi:","Se SIM para a PERGUNTA 29, existe envolvimento...","Se SIM para a PERGUNTA 29, existe envolvimento...",Em relação à produção de relatos baseados na i...,Em relação à produção de relatos baseados na i...,Em relação à produção de relatos baseados na i...,Em relação à produção de relatos baseados na i...
alternativas,Sim,Não,"Na Visão, ou declaração de natureza semelhante","Na Missão, ou declaração de natureza semelhante",No Código de Conduta que explicite valores e p...,Nas políticas da organização,Nos objetivos e metas de desempenho socioambie...,Nenhuma das anteriores,Respeito ao meio ambiente,Práticas leais de operação,...,Integralmente disponibilizado em pelo menos um...,Divulgado em versão adaptada especificamente a...,"Divulgado em versões adicionais, que visem à s...",Nenhuma das anteriores,Sim,Não,"Estuda o tema, porém ainda não adotou medidas ...",Adotou medidas para publicação de relatos que ...,Já publicou relatos com essas características,Nenhuma das anteriores


,0,0,1,1,1,1,1,1,2,2,...,62,62,62,62,63,63,64,64,64,64
enunciado,O compromisso com o <strong>Desenvolvimento Su...,O compromisso com o <strong>Desenvolvimento Su...,"Se SIM para a <span>PERGUNTA 1</span>, este co...","Se SIM para a <span>PERGUNTA 1</span>, este co...","Se SIM para a <span>PERGUNTA 1</span>, este co...","Se SIM para a <span>PERGUNTA 1</span>, este co...","Se SIM para a <span>PERGUNTA 1</span>, este co...","Se SIM para a <span>PERGUNTA 1</span>, este co...","Se SIM para a <span>PERGUNTA 1</span>, assinal...","Se SIM para a <span>PERGUNTA 1</span>, assinal...",...,"Se SIM para a<span> PERGUNTA 29</span>, o rela...","Se SIM para a<span> PERGUNTA 29</span>, o rela...","Se SIM para a<span> PERGUNTA 29</span>, o rela...","Se SIM para a<span> PERGUNTA 29</span>, o rela...","Se SIM para a <span>PERGUNTA 29</span>, existe...","Se SIM para a <span>PERGUNTA 29</span>, existe...",Em relação à produção de relatos baseados na i...,Em relação à produção de relatos baseados na i...,Em relação à produção de relatos baseados na i...,Em relação à produção de relatos baseados na i...
alternativas,Sim,Não,"Na Visão, ou declaração de natureza semelhante","Na Missão, ou declaração de natureza semelhante",No <strong>Código de Conduta</strong> que expl...,Nas políticas da organização,Nos objetivos e <strong>metas de desempenho so...,Nenhuma das anteriores,Respeito ao meio ambiente,Práticas leais de operação,...,Integralmente disponibilizado em pelo menos um...,Divulgado em <strong>versão adaptada</strong> ...,"Divulgado em versões adicionais, que visem à s...",Nenhuma das anteriores,Sim,Não,"Estuda o tema, porém ainda não adotou medidas ...",Adotou medidas para publicação de relatos que ...,Já publicou relatos com essas características,Nenhuma das anteriores
answer,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


['A', 'C']